## Готовимся к машинному обучению

Для начала вчитаем информацию по нашему подготовленному тренировочному множеству.

In [1]:
import csv, gzip, os, glob
import numpy, scipy, pandas
from scipy.sparse import coo_matrix, csr_matrix

In [2]:
dataPath = "./"

In [3]:
loaded = numpy.load(os.path.join(dataPath,"commonFriends_train.npz"))
commonFriends_train = csr_matrix((loaded['data'], loaded['indices'], loaded['indptr']), shape=loaded['shape']).tocoo()

In [4]:
loaded = numpy.load(os.path.join(dataPath,"train_markup.npz"))
train_markup = csr_matrix((loaded['data'], loaded['indices'], loaded['indptr']), shape=loaded['shape']).tocoo()

In [5]:
del loaded

## Запускаем панду

Для того чтобы удобно было мержить данные, преобразуем их в пандовский датафреймы - таблицы, хранящие данные по массиву на колонку. Для это при чтении переводили матрицы в coo формат

In [6]:
train_markup_df = pandas.DataFrame(
    data = {"from" : train_markup.row, "to" : train_markup.col, "label" : train_markup.data})
train_markup_df

,from,label,to
0,179,1,10223
1,179,1,18608
2,179,1,44314
3,179,1,50110
4,179,1,104087
5,179,1,125634
6,179,1,164651
7,179,1,167411
8,179,1,191007
9,179,1,198985


Вывод в вебморде замечательный :) Теперь втянем счетчики общих друзей

In [8]:
train_common_friends_df = pandas.DataFrame(
    data = {"from" : commonFriends_train.row, "to" : commonFriends_train.col, "common_neighbors" : commonFriends_train.data})
train_common_friends_df

,common_neighbors,from,to
0,1,179,2772608
1,1,179,14466587
2,1,179,8622830
3,1,179,15904467
4,1,179,4450363
5,1,179,16386542
6,1,179,16202341
7,1,179,15117134
8,1,179,14775137
9,1,179,14518169


## Балансируем

51М пар с общими друзьями, а в графе всего 3м - датасет несбалансирован. В таких случаях полезно бывает засэмплить негативные примеры. Но сначала соберем позитивы.

In [9]:
positives = pandas.merge(
    train_markup_df,
    train_common_friends_df,
    how='inner',
    left_on = ["from","to"],
    right_on = ["from","to"])
positives

,from,label,to,common_neighbors
0,179,1,2545686,22
1,179,1,4033722,13
2,179,1,7277379,5
3,179,1,8001525,4
4,179,1,10958460,23
5,179,1,11525540,33
6,179,1,13570262,8
7,179,1,13822924,28
8,179,1,13987529,23
9,179,1,14064795,5


Почему позитивов оказалось только 423к? Наосячили с выборкой? Нет - просто общие друзья подсчитаны только между пользователями ядра, а в графе были и внешние их связи. Теперь понятно что для негативов нада сэмплировать около 1% данных 

In [10]:
sample_neg = train_common_friends_df.sample(frac=0.01, replace=True)

Сразу вычистим из памяти здоровущий кусок. А затем сделаем лефт джоин, оставим только налы и заменим их на 0.

In [11]:
del train_common_friends_df

In [12]:
pre_negatives = pandas.merge(
    sample_neg,
    train_markup_df,
    how='left',
    left_on = ["from","to"],
    right_on = ["from","to"])

In [13]:
pre_negatives

,common_neighbors,from,to,label
0,3,11733406,10173690,NaN
1,1,722329,4823863,NaN
2,6,5188307,6482443,NaN
3,1,6309405,3663406,NaN
4,3,11179094,10614982,NaN
5,1,5710290,4074918,NaN
6,2,5651814,332652,NaN
7,1,13194877,7516895,NaN
8,1,15087075,2698086,NaN
9,2,4495439,1013278,NaN


In [14]:
negatives = pre_negatives[pre_negatives["label"] != 1].replace(float("NaN"), 0.0)
negatives

,common_neighbors,from,to,label
0,3,11733406,10173690,0.0
1,1,722329,4823863,0.0
2,6,5188307,6482443,0.0
3,1,6309405,3663406,0.0
4,3,11179094,10614982,0.0
5,1,5710290,4074918,0.0
6,2,5651814,332652,0.0
7,1,13194877,7516895,0.0
8,1,15087075,2698086,0.0
9,2,4495439,1013278,0.0


Последний штрих - объединяем позитивы с негативами и получаем миллионный датасет с 1-й фичей :)

In [15]:
dataset = pandas.concat([positives,negatives])
dataset

,common_neighbors,from,label,to
0,22,179,1.0,2545686
1,13,179,1.0,4033722
2,5,179,1.0,7277379
3,4,179,1.0,8001525
4,23,179,1.0,10958460
5,33,179,1.0,11525540
6,8,179,1.0,13570262
7,28,179,1.0,13822924
8,23,179,1.0,13987529
9,5,179,1.0,14064795


Остальное нам теперь не нужно

In [16]:
del positives
del negatives
del sample_neg
del train_markup_df

## Подключаем демографию

1 фича это маловато. Где взять еще? Очевидный вариант - из демографии пользователей. Нам повезло - панда умеет читать и csv :)

In [17]:
core = pandas.DataFrame()

# Итерируемся по файлам
for f in glob.glob("./coreDemography/part*"):
    # Собираем все в один датафрэйм
    core = pandas.concat(
        [core,
         # Поддержка csv из коробки :)
        pandas.read_csv(
        f, 
        sep = "\t",
        # Поддержка gzip из коробки :)
        compression = 'gzip',
        # Заводим индекс - будет быстрый доступ по ИД юзера
        index_col = ["userId"],
        names = ["userId", "create_date", "birth_date", "gender", "ID_country", "ID_Location", "loginRegion"],
        dtype = {
                "userId" : numpy.int32,
                "create_date" : numpy.int64,
                "birth_date" : numpy.float16,
                "gender" : numpy.int8,
                "ID_country" : numpy.int64,
                "ID_Location" : numpy.int64,
                # Поскольку поле может быть пусто, используем float. На месте пустоты будет NaN
                "loginRegion" : numpy.float32
                }
        )])

# Ну а вывод на консоль совсем зачотный
core

,create_date,birth_date,gender,ID_country,ID_Location,loginRegion
userId,,,,,,
16460783,1182885174073,486.0,2,10414533690,1078547,85.0
16467391,1176953226317,4668.0,2,10414533690,1384327,85.0
16467889,1169816093060,6860.0,2,10414533690,33438,NaN
16468013,1172074147460,5360.0,2,10415971874,4000749,1.0
16471027,1182034019343,6312.0,2,10414533690,3047069,11.0
16474162,1173115608560,5568.0,2,10414533690,3868434,36.0
16476386,1151057658677,4352.0,2,10414533690,3385314,11.0
16481067,1177305838543,3832.0,2,10414533690,1112209,13.0
16485965,1178801047573,-2368.0,2,10415971874,2225871,96.0


Замечательно! Теперь делаем джоин демографии с основным датасетом. Два раза. Потому как from и to.

In [18]:
withDemography = pandas.merge(
    pandas.merge(
        dataset,
        core,
        how='inner',
        left_on = ["from"],
        right_index = True),
    core,
    how='inner',
    left_on = ["to"],
    right_index = True,
    suffixes = ['',"_to"])
withDemography

,common_neighbors,from,label,to,create_date,birth_date,gender,ID_country,ID_Location,loginRegion,create_date_to,birth_date_to,gender_to,ID_country_to,ID_Location_to,loginRegion_to
0,22,179,1.0,2545686,1183207776777,-64.0,1,10414533690,827938,43.0,1241009554407,1763.0,1,10414533690,4599571,23.0
376330,2,296398,0.0,2545686,1199721065610,5512.0,1,10414533690,1078547,85.0,1241009554407,1763.0,1,10414533690,4599571,23.0
4293,1,374036,0.0,2545686,1198873312280,-4492.0,2,10414533690,1078547,85.0,1241009554407,1763.0,1,10414533690,4599571,23.0
66341,4,1389842,0.0,2545686,1224073888587,5640.0,2,10414533690,3385314,NaN,1241009554407,1763.0,1,10414533690,4599571,23.0
337320,1,1859520,0.0,2545686,1237907243513,-3018.0,2,10414533690,2407939,26.0,1241009554407,1763.0,1,10414533690,4599571,23.0
72220,2,2388048,1.0,2545686,1255363239223,3684.0,2,10414533690,3385314,NaN,1241009554407,1763.0,1,10414533690,4599571,23.0
122703,1,9717986,0.0,2545686,1348059475107,7416.0,2,10414533690,2007580,2.0,1241009554407,1763.0,1,10414533690,4599571,23.0
406105,1,11108463,0.0,2545686,1357211578650,4556.0,2,10415971874,1832332,59.0,1241009554407,1763.0,1,10414533690,4599571,23.0
333649,183,12802551,1.0,2545686,1398158518590,1872.0,1,10424076448,1722158,33.0,1241009554407,1763.0,1,10414533690,4599571,23.0
350758,47,13634932,1.0,2545686,1415986935390,4184.0,1,10414533690,1431892,5.0,1241009554407,1763.0,1,10414533690,4599571,23.0


Удачно получилось - для всех юзеров нашлась демография. Сохраним файлик для дальнейшей обработки в csv

In [19]:
withDemography.to_csv("./commonNeigborsAndDemography.csv")

# Главный принцип машинного обучения

__Многие задачи хорошо решаются и без машинного обучения.__